# Assigment 8

1. Each group must create their branch named group_#_ass_8_2022 (group_1_ass_8_2022) and save their results in the [Assignment_8 folder](https://github.com/alexanderquispe/Diplomado_PUCP/tree/main/Lecture_9/Assignment_8). Name your JN like your branch.
2. Import the data located at this [link](https://github.com/alexanderquispe/Diplomado_PUCP/blob/main/_data/cajamarca_data_helth_estb.xlsx). This dataset comes from this [link](https://www.datosabiertos.gob.pe/dataset/establecimientos-de-salud). It has information on all the geolocated health establishments in Cajamarca.
3. Import the [district's centroids data](https://github.com/alexanderquispe/Diplomado_PUCP/blob/main/_data/peru_districts_centroids.xlsx). It has the latitude and longitude of every district in Peru.
4. Use these datasets and find the closest health establishment to Cajamarca district centroids. Use euclidian distance between points. DO NOT USE GOOGLE API. IT WILL BE VERY EXPENSIVE For each district, there should be only one health establishment.
5. Use Google API Directions to find the driving travel time and distance from health establishments to district centroids. Set driving as the value of the parameter mode and Monday 4pm as departure_time. Include information on the three options of the traffic_model variable: best_guess, pessimistic, and optimistic. The final data frame output should look like the table below:
    1. |id_eess| 
    2. |codigo_renaes| 
    3. |categoria| 
    4. |nombre| 
    5. |diresa|	 
    6. |red|
    7. |direccion|
    8. |longitud|
    9. |latitud|
    10. |id_ubigeo|
    11. |UBIGEO|
    12. |CCDD|
    13. |NOMBDEP|	
    14. |CCPP|	
    15. |NOMBPROV|	
    16. |CCDI|	
    17. |NOMBDIST|	
    18. |CAPITAL|	
    19. |District|	
    20. |Dist_Centroid_Latitude|	
    21. |Dist_Centroid_Longitude|	
    22. |travel_time_best_guess|	
    23. |travel_time_pessimistic|	
    24. |travel_time_optimistic|	
    25. |travel_distance_best_guess|	
    26. |travel_distance_pessimistic|	
    27. |travel_distance_optimistic|
6. Make a histogram of travel time colored by hue = traffic_model (best_guess, pessimistic, and optimistic) for each Province..
7. Make a histogram of travel distance colored by hue = traffic_model (best_guess, pessimistic, and optimistic) for each Province.
8. Bar plot of the mean travel distance by Province colored by hue = traffic_model (best_guess, pessimistic, and optimistic). The x-axis should be the name of the Province.
9. Bar plot of the mean travel time by Province colored by hue = traffic_model (best_guess, pessimistic, and optimistic). The x-axis should be the name of the Province.
10. Make plots 10 and 11 in just one figure. Hint: Use catplot. Use [this data](https://github.com/alexanderquispe/Diplomado_PUCP/blob/main/_data/salud_places_peru.csv) for the following plot.
11. Make a count plot of the number of health establishments by Department colored by hue = categoria. The x-axis should be the Department Name.


In [4]:
# Librería necesarias para el trabajo
import tabula
import numpy as np
import pandas as pd
import os
import urllib.request, json, csv
import numpy as np
from tqdm import tqdm_notebook as tqdm
import requests
import googlemaps
from datetime import datetime

In [5]:
pwd

'D:\\GitHub\\Diplomado_PUCP\\Lecture_9\\Assignment_8'

In [6]:
# Importar la data geolocalizada de los centros de salud de Cajamarca
data = pd.read_excel("https://github.com/alexanderquispe/Diplomado_PUCP/raw/main/_data/cajamarca_data_helth_estb.xlsx")
data

,id_eess,codigo_renaes,categoria,nombre,diresa,red,direccion,longitud,latitud,id_ubigeo,...,CCPP,NOMBPROV,CCDI,NOMBDIST,CAPITAL,UBIGEO,IDPROV,CODIGO,CNT_CCPP,DESCRIPCIO
0,9439,4210,II-1,GRAL. JAEN,CAJAMARCA,NO PERTENECE A NINGUNA RED,AVENIDA AV. PAKAMAUROS S/N S/N AV. PAKAMAUROS ...,-78.804711,-5.706214,636,...,8,JAEN,1,JAEN,JAEN,60801,608,60801,108,JAEN
1,27242,4211,I-4,MORRO SOLAR,CAJAMARCA,JAEN,OTROS ALFREDO BASTOS Nº 630 ALFREDO BASTOS Nº ...,-78.811920,-5.710600,636,...,8,JAEN,1,JAEN,JAEN,60801,608,60801,108,JAEN
2,22911,4212,I-3,MAGLLANAL,CAJAMARCA,JAEN,CALLE CALLE LAS ALMENDRA CDRA. 4 S/N S/N CALLE...,-78.817266,-5.707647,636,...,8,JAEN,1,JAEN,JAEN,60801,608,60801,108,JAEN
3,9242,4213,I-3,FILA ALTA,CAJAMARCA,JAEN,OTROS COMUNIDAD FILA ALTA COMUNIDAD FILA ALTA ...,-78.791643,-5.739184,636,...,8,JAEN,1,JAEN,JAEN,60801,608,60801,108,JAEN
4,26418,4214,I-1,LAS NARANJAS,CAJAMARCA,JAEN,OTROS COMUNIDAD DE LAS NARANJAS COMUNIDAD DE L...,-78.846347,-5.745295,636,...,8,JAEN,1,JAEN,JAEN,60801,608,60801,108,JAEN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187,21625,5038,I-3,SAN LUIS DE LA LUCMA,CAJAMARCA,SOCOTA,CARRETERA CARRETERA LUCMA SOCOTA DISTRITO SAN ...,-78.607471,-6.292044,627,...,6,CUTERVO,10,SAN LUIS DE LUCMA,SAN LUIS DE LUCMA,60610,606,60610,19,SAN LUIS DE LUCMA
1188,8179,6853,I-1,SAIREPAMPA,CAJAMARCA,SOCOTA,OTROS COMUNIDAD DE SAIREPAMPA DISTRITO SAN LUI...,-78.631080,-6.279960,627,...,6,CUTERVO,10,SAN LUIS DE LUCMA,SAN LUIS DE LUCMA,60610,606,60610,19,SAN LUIS DE LUCMA
1189,16451,6870,I-1,SANTA ROSA DEL TINGO,CAJAMARCA,SOCOTA,OTROS COMUNIDAD DE SANTA ROSA DEL TINGO DISTRI...,-78.597640,-6.266560,627,...,6,CUTERVO,10,SAN LUIS DE LUCMA,SAN LUIS DE LUCMA,60610,606,60610,19,SAN LUIS DE LUCMA
1190,3040,7048,I-1,SEXESHITA,CAJAMARCA,SOCOTA,OTROS COMUNIDAD DE SEXESHITA DISTRITO SAN LUIS...,-78.627210,-6.267210,627,...,6,CUTERVO,10,SAN LUIS DE LUCMA,SAN LUIS DE LUCMA,60610,606,60610,19,SAN LUIS DE LUCMA


In [7]:
# Importar la data de los centroides de los distritos
data_centroides = pd.read_excel("https://github.com/alexanderquispe/Diplomado_PUCP/raw/main/_data/peru_districts_centroids.xlsx")
data_centroides

,CCDD,NOMBDEP,CCPP,NOMBPROV,CCDI,NOMBDIST,CAPITAL,UBIGEO,IDPROV,CODIGO,Centroid_Latitude,Centroid_Longitude
0,6,CAJAMARCA,5,CONTUMAZA,4,GUZMANGO,GUZMANGO,60504,605,60504,-7.364304,-78.917689
1,6,CAJAMARCA,11,SAN MIGUEL,5,EL PRADO,EL PRADO,61105,611,61105,-7.038039,-79.003318
2,6,CAJAMARCA,11,SAN MIGUEL,9,NIEPOS,NIEPOS,61109,611,61109,-6.926718,-79.131435
3,6,CAJAMARCA,11,SAN MIGUEL,10,SAN GREGORIO,SAN GREGORIO,61110,611,61110,-7.117978,-79.160968
4,6,CAJAMARCA,12,SAN PABLO,3,SAN LUIS,SAN LUIS GRANDE,61203,612,61203,-7.159046,-78.873084
...,...,...,...,...,...,...,...,...,...,...,...,...
1869,4,AREQUIPA,4,CASTILLA,4,CHACHAS,CHACHAS,40404,404,40404,-15.181923,-72.116054
1870,4,AREQUIPA,4,CASTILLA,5,CHILCAYMARCA,CHILCAYMARCA,40405,404,40405,-15.350613,-72.385276
1871,4,AREQUIPA,4,CASTILLA,6,CHOCO,CHOCO,40406,404,40406,-15.414004,-72.068684
1872,4,AREQUIPA,4,CASTILLA,7,HUANCARQUI,HUANCARQUI,40407,404,40407,-16.012364,-72.368167


In [8]:
# Revisamos el nombre de las variables de las dos bases de datos
print(data.columns) 
print(data_centroides.columns)

Index(['id_eess', 'codigo_renaes', 'categoria', 'nombre', 'diresa', 'red',
       'direccion', 'longitud', 'latitud', 'id_ubigeo', 'ind', 'CCDD',
       'NOMBDEP', 'CCPP', 'NOMBPROV', 'CCDI', 'NOMBDIST', 'CAPITAL', 'UBIGEO',
       'IDPROV', 'CODIGO', 'CNT_CCPP', 'DESCRIPCIO'],
      dtype='object')
Index(['CCDD', 'NOMBDEP', 'CCPP', 'NOMBPROV', 'CCDI', 'NOMBDIST', 'CAPITAL',
       'UBIGEO', 'IDPROV', 'CODIGO', 'Centroid_Latitude',
       'Centroid_Longitude'],
      dtype='object')


In [10]:
# Eliminando las variables innecesarias y revisando las columnas
data_2 = data.drop(["ind","IDPROV","CNT_CCPP","DESCRIPCIO"], axis = 1)
data_2.columns

Index(['id_eess', 'codigo_renaes', 'categoria', 'nombre', 'diresa', 'red',
       'direccion', 'longitud', 'latitud', 'id_ubigeo', 'CCDD', 'NOMBDEP',
       'CCPP', 'NOMBPROV', 'CCDI', 'NOMBDIST', 'CAPITAL', 'UBIGEO', 'CODIGO'],
      dtype='object')

In [11]:
# Eligiendo las varaibles necesarias a fusionar con el DataFrame previo
data_centroides_2 = data_centroides.loc[ : , ["CODIGO","Centroid_Latitude","Centroid_Longitude"]]
data_centroides_2

,CODIGO,Centroid_Latitude,Centroid_Longitude
0,60504,-7.364304,-78.917689
1,61105,-7.038039,-79.003318
2,61109,-6.926718,-79.131435
3,61110,-7.117978,-79.160968
4,61203,-7.159046,-78.873084
...,...,...,...
1869,40404,-15.181923,-72.116054
1870,40405,-15.350613,-72.385276
1871,40406,-15.414004,-72.068684
1872,40407,-16.012364,-72.368167


In [12]:
# Generamos un objeto que fusiones los datos de los dataframe previos
merged_data = data_2.merge(data_centroides_2, on='CODIGO', how='inner')
merged_data

,id_eess,codigo_renaes,categoria,nombre,diresa,red,direccion,longitud,latitud,id_ubigeo,...,NOMBDEP,CCPP,NOMBPROV,CCDI,NOMBDIST,CAPITAL,UBIGEO,CODIGO,Centroid_Latitude,Centroid_Longitude
0,9439,4210,II-1,GRAL. JAEN,CAJAMARCA,NO PERTENECE A NINGUNA RED,AVENIDA AV. PAKAMAUROS S/N S/N AV. PAKAMAUROS ...,-78.804711,-5.706214,636,...,CAJAMARCA,8,JAEN,1,JAEN,JAEN,60801,60801,-5.770179,-78.852551
1,27242,4211,I-4,MORRO SOLAR,CAJAMARCA,JAEN,OTROS ALFREDO BASTOS Nº 630 ALFREDO BASTOS Nº ...,-78.811920,-5.710600,636,...,CAJAMARCA,8,JAEN,1,JAEN,JAEN,60801,60801,-5.770179,-78.852551
2,22911,4212,I-3,MAGLLANAL,CAJAMARCA,JAEN,CALLE CALLE LAS ALMENDRA CDRA. 4 S/N S/N CALLE...,-78.817266,-5.707647,636,...,CAJAMARCA,8,JAEN,1,JAEN,JAEN,60801,60801,-5.770179,-78.852551
3,9242,4213,I-3,FILA ALTA,CAJAMARCA,JAEN,OTROS COMUNIDAD FILA ALTA COMUNIDAD FILA ALTA ...,-78.791643,-5.739184,636,...,CAJAMARCA,8,JAEN,1,JAEN,JAEN,60801,60801,-5.770179,-78.852551
4,26418,4214,I-1,LAS NARANJAS,CAJAMARCA,JAEN,OTROS COMUNIDAD DE LAS NARANJAS COMUNIDAD DE L...,-78.846347,-5.745295,636,...,CAJAMARCA,8,JAEN,1,JAEN,JAEN,60801,60801,-5.770179,-78.852551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187,21625,5038,I-3,SAN LUIS DE LA LUCMA,CAJAMARCA,SOCOTA,CARRETERA CARRETERA LUCMA SOCOTA DISTRITO SAN ...,-78.607471,-6.292044,627,...,CAJAMARCA,6,CUTERVO,10,SAN LUIS DE LUCMA,SAN LUIS DE LUCMA,60610,60610,-6.279101,-78.578753
1188,8179,6853,I-1,SAIREPAMPA,CAJAMARCA,SOCOTA,OTROS COMUNIDAD DE SAIREPAMPA DISTRITO SAN LUI...,-78.631080,-6.279960,627,...,CAJAMARCA,6,CUTERVO,10,SAN LUIS DE LUCMA,SAN LUIS DE LUCMA,60610,60610,-6.279101,-78.578753
1189,16451,6870,I-1,SANTA ROSA DEL TINGO,CAJAMARCA,SOCOTA,OTROS COMUNIDAD DE SANTA ROSA DEL TINGO DISTRI...,-78.597640,-6.266560,627,...,CAJAMARCA,6,CUTERVO,10,SAN LUIS DE LUCMA,SAN LUIS DE LUCMA,60610,60610,-6.279101,-78.578753
1190,3040,7048,I-1,SEXESHITA,CAJAMARCA,SOCOTA,OTROS COMUNIDAD DE SEXESHITA DISTRITO SAN LUIS...,-78.627210,-6.267210,627,...,CAJAMARCA,6,CUTERVO,10,SAN LUIS DE LUCMA,SAN LUIS DE LUCMA,60610,60610,-6.279101,-78.578753


In [13]:
# Revisamos las variables en la DataFrame fusionada y si hay valores perdidos
print(merged_data.columns)
print(merged_data.isnull().sum())
# Se comprueba que no hay valores perdidos

Index(['id_eess', 'codigo_renaes', 'categoria', 'nombre', 'diresa', 'red',
       'direccion', 'longitud', 'latitud', 'id_ubigeo', 'CCDD', 'NOMBDEP',
       'CCPP', 'NOMBPROV', 'CCDI', 'NOMBDIST', 'CAPITAL', 'UBIGEO', 'CODIGO',
       'Centroid_Latitude', 'Centroid_Longitude'],
      dtype='object')
id_eess               0
codigo_renaes         0
categoria             0
nombre                0
diresa                0
red                   0
direccion             0
longitud              0
latitud               0
id_ubigeo             0
CCDD                  0
NOMBDEP               0
CCPP                  0
NOMBPROV              0
CCDI                  0
NOMBDIST              0
CAPITAL               0
UBIGEO                0
CODIGO                0
Centroid_Latitude     0
Centroid_Longitude    0
dtype: int64


In [14]:
# importamos la libreria necesaria para calcular la distancia euclediana
from scipy.spatial import distance

# Creamos una función para calcular la distancia euclidiana
def distancia_euclidiana(fila):
    coord1 = (fila['latitud'], fila['longitud'])
    coord2 = (fila['Centroid_Latitude'], fila['Centroid_Longitude'])
    return distance.euclidean(coord1, coord2)

# Creamos una nueva columna en el DataFrame con la distancia euclidiana
merged_data["distancia"] = merged_data.apply(distancia_euclidiana, axis=1)
merged_data

,id_eess,codigo_renaes,categoria,nombre,diresa,red,direccion,longitud,latitud,id_ubigeo,...,CCPP,NOMBPROV,CCDI,NOMBDIST,CAPITAL,UBIGEO,CODIGO,Centroid_Latitude,Centroid_Longitude,distancia
0,9439,4210,II-1,GRAL. JAEN,CAJAMARCA,NO PERTENECE A NINGUNA RED,AVENIDA AV. PAKAMAUROS S/N S/N AV. PAKAMAUROS ...,-78.804711,-5.706214,636,...,8,JAEN,1,JAEN,JAEN,60801,60801,-5.770179,-78.852551,0.079875
1,27242,4211,I-4,MORRO SOLAR,CAJAMARCA,JAEN,OTROS ALFREDO BASTOS Nº 630 ALFREDO BASTOS Nº ...,-78.811920,-5.710600,636,...,8,JAEN,1,JAEN,JAEN,60801,60801,-5.770179,-78.852551,0.072114
2,22911,4212,I-3,MAGLLANAL,CAJAMARCA,JAEN,CALLE CALLE LAS ALMENDRA CDRA. 4 S/N S/N CALLE...,-78.817266,-5.707647,636,...,8,JAEN,1,JAEN,JAEN,60801,60801,-5.770179,-78.852551,0.071800
3,9242,4213,I-3,FILA ALTA,CAJAMARCA,JAEN,OTROS COMUNIDAD FILA ALTA COMUNIDAD FILA ALTA ...,-78.791643,-5.739184,636,...,8,JAEN,1,JAEN,JAEN,60801,60801,-5.770179,-78.852551,0.068341
4,26418,4214,I-1,LAS NARANJAS,CAJAMARCA,JAEN,OTROS COMUNIDAD DE LAS NARANJAS COMUNIDAD DE L...,-78.846347,-5.745295,636,...,8,JAEN,1,JAEN,JAEN,60801,60801,-5.770179,-78.852551,0.025645
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187,21625,5038,I-3,SAN LUIS DE LA LUCMA,CAJAMARCA,SOCOTA,CARRETERA CARRETERA LUCMA SOCOTA DISTRITO SAN ...,-78.607471,-6.292044,627,...,6,CUTERVO,10,SAN LUIS DE LUCMA,SAN LUIS DE LUCMA,60610,60610,-6.279101,-78.578753,0.031500
1188,8179,6853,I-1,SAIREPAMPA,CAJAMARCA,SOCOTA,OTROS COMUNIDAD DE SAIREPAMPA DISTRITO SAN LUI...,-78.631080,-6.279960,627,...,6,CUTERVO,10,SAN LUIS DE LUCMA,SAN LUIS DE LUCMA,60610,60610,-6.279101,-78.578753,0.052334
1189,16451,6870,I-1,SANTA ROSA DEL TINGO,CAJAMARCA,SOCOTA,OTROS COMUNIDAD DE SANTA ROSA DEL TINGO DISTRI...,-78.597640,-6.266560,627,...,6,CUTERVO,10,SAN LUIS DE LUCMA,SAN LUIS DE LUCMA,60610,60610,-6.279101,-78.578753,0.022672
1190,3040,7048,I-1,SEXESHITA,CAJAMARCA,SOCOTA,OTROS COMUNIDAD DE SEXESHITA DISTRITO SAN LUIS...,-78.627210,-6.267210,627,...,6,CUTERVO,10,SAN LUIS DE LUCMA,SAN LUIS DE LUCMA,60610,60610,-6.279101,-78.578753,0.049895


In [19]:
# Identificamos la lista de distritos que existen en Cajamarca
valores_columna = merged_data['NOMBDIST'].unique()
print(valores_columna)

['JAEN' 'BELLAVISTA' 'CHONTALI' 'COLASAY' 'HUABAL' 'SAN JOSE DEL ALTO'
 'LAS PIRIAS' 'POMAHUACA' 'PUCARA' 'SALLIQUE' 'SAN FELIPE' 'SANTA ROSA'
 'SAN IGNACIO' 'NAMBALLE' 'CHIRINOS' 'HUARANGO' 'LA COIPA'
 'SAN JOSE DE LOURDES' 'TABACONAS' 'CELENDIN' 'UTCO' 'SOROCHUCO' 'HUASMIN'
 'CORTEGANA' 'PACCHA' 'MIGUEL IGLESIAS' 'CHUMUCH' 'LA LIBERTAD DE PALLAN'
 'SUCRE' 'OXAMARCA' 'JOSE GALVEZ' 'JORGE CHAVEZ' 'ICHOCAN' 'CHANCAY'
 'EDUARDO VILLANUEVA' 'JOSE MANUEL QUIROZ' 'JOSE SABOGAL' 'PEDRO GALVEZ'
 'GREGORIO PITA' 'MATARA' 'CAJABAMBA' 'CACHACHI' 'CONDEBAMBA' 'SITACOCHA'
 'NANCHOC' 'BOLIVAR' 'SAN GREGORIO' 'CHILETE' 'SAN BERNARDINO' 'EL PRADO'
 'SAN MIGUEL' 'UNION AGUA BLANCA' 'TANTARICA' 'CONTUMAZA'
 'SANTA CRUZ DE TOLED' 'SAN BENITO' 'CASCAS' 'GUZMANGO' 'LA FLORIDA'
 'NIEPOS' 'LLAPA' 'SAN SILVESTRE DE COCHAN' 'CALQUIS' 'SAN PABLO'
 'TUMBADEN' 'SAN LUIS' 'YONAN' 'CUPISNIQUE' 'LOS BAÑ\x91OS DEL INCA'
 'ENCAÑ\x91ADA' 'NAMORA' 'LLACANORA' 'CATILLUC' 'TONGOD' 'CAJAMARCA'
 'ASUNCION' 'COSPAN' 'MAGDAL